In [19]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

## Fetch data

In [20]:
re = requests.get('https://guide.berkeley.edu/undergraduate/degree-programs/')
soup = BeautifulSoup(re.text)
lst = soup.find('ul',{'class':'program-thumb-list'}).find_all('li')

url = 'https://guide.berkeley.edu/undergraduate/degree-programs/'
link_dic = dict()
for li in lst:
  major_name = li.find('a')['href']
  link_dic[major_name] = url + major_name

soup_dic = dict()
for key in link_dic:
  re = requests.get(link_dic[key] + '/')
  soup_dic[key] = BeautifulSoup(re.text)

## Structure EDA

In [47]:
max = 0
major_dic = dict()
for key in soup_dic:
  li_table = soup_dic[key].find('ul',{'class':'clearfix'})
  if li_table:

    li_table = li_table.find_all('li')
    if len(li_table) > max:

      max = len(li_table)

      for each_lst in major_dic.values(): #appending na to the other lists less than max
        while len(each_lst) < max:
          each_lst.append(np.nan)

      major_dic[key] = list()
      for li in li_table:
        major_dic[key].append(li['id'])

    else:
      major_dic[key] = list()
      for li in li_table:
        major_dic[key].append(li['id'])

      while len(major_dic[key]) < max:
          major_dic[key].append(np.nan)

  else:
    continue


In [48]:
df = pd.DataFrame.from_dict(major_dic, orient='index')

In [66]:
condition = (df[3]=='studentlearninggoalstexttab') | (df[4]=='studentlearninggoalstexttab') | (df[5]=='studentlearninggoalstexttab') | (df[6]=='studentlearninggoalstexttab')
filtered = df[condition]

## Scrapping only majors with learning outcome

In [84]:
learning_major = dict()
for key in soup_dic:
  if key in filtered.index.to_list():
    string = ''
    tags = soup_dic[key].find('div',{'aria-labelledby':'studentlearninggoalstexttab'})
    for tag in list(tags):
      string += tag.text
      string += '\n'
      learning_major[key] = [string]

In [87]:
new_df = pd.DataFrame.from_dict(learning_major, orient='index')

In [93]:
new_df.to_json('corpus,json')